<a href="https://colab.research.google.com/github/FCUAIC/Basic-ML/blob/main/mnist_plus_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Plus Template

作者: 梁定殷

版權歸FCUAI所有

Input shape: (1, 28, 28)

## 超參數(Hyperparameters)

In [ ]:
# 學習率(Learning Rate), LR越大模型越有自信, LR越小模型越沒自信
LR = 0.01
# 每次學習要看過多少的Batch後才更新模型
BATCH_SIZE = 1024
# 學習次數
EPOCHS = 10

用離線的MNIST = True # Pytorch的MNIST暫時不能用

## 載入需要用的Package

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adadelta
from torch.nn import CrossEntropyLoss
from torchsummary import summary

from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

if 用離線的MNIST:
    from keras.datasets import mnist
else:
    from torchvision.datasets import MNIST


from sklearn.model_selection import  train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

from tqdm.notebook import trange, tqdm

## 模型

In [ ]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.cnn = nn.Sequential(
            # in: 1x28x28
            nn.Conv2d(in_channels=1,
                      out_channels=32,
                      kernel_size=3,
                      stride=1,
                      padding=?
                    ),
            nn.ReLU(),
            # in: 32x?x?
            nn.MaxPool2D(?),

            # in: 32x?x?
            nn.Conv2d(in_channels=32,
                      out_channels=64,
                      kernel_size=3,
                      stride=1,
                      padding=?
                    ),
            nn.Dropout(),
            nn.ReLU(),
            # in: 64x?x?
            nn.Flatten(),
            # out: ?
        )

        self.fc = nn.Sequential(
            nn.Linear(in_features=?, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=10),
        )
    
    def forward(self, x):
        x = self.cnn(x)
        return self.fc(x)

model = MNISTModel().cuda()
summary(model, (1, 28, 28))

## 資料集(Dataset)

In [ ]:
class MNISTDataset(Dataset):
    """
        這是我們定義的Dataset
    """
    def __init__(self, train=False, transformer=None):
        # 從Keras載入MNIST
        (train_feature, train_label), (test_feature, test_label) = mnist.load_data()
        if train:
            # 我們只要訓練的
            self.data = np.array([list(d) for d in zip(train_feature, train_label)], dtype=object)
            self.length = len(train_feature)
        else:
            # 我們只要測試的
            self.data = np.array([list(d) for d in zip(test_feature, test_label)], dtype=object)
            self.length = len(test_feature)
        
        self.transformer = transformer

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # 對data做轉換
        if self.transformer:
            img, label = self.data[idx]
            return self.transformer(img), torch.tensor(label, dtype=torch.long)
        return self.data[idx]

## 載入資料集(Dataset)

In [ ]:
preprocessor = transforms.Compose([
    transforms.ToTensor() #轉成Tensor的時候會做歸一化(Normalize)
    ])


if 用離線的MNIST:
    print('使用離線的Dataset.')
    mnist_train = MNISTDataset(train=True, transformer=preprocessor)
    mnist_test = MNISTDataset(train=False, transformer=preprocessor)
else:
    print('使用Pytorch的Dataset.')
    mnist_train = MNIST(root='mnist', download=True, transform=preprocessor, train=True)
    mnist_test = MNIST(root='mnist', transform=preprocessor, train=False)

print(f'訓練資料一共有{len(mnist_train)}筆資料\n測試資料一共有{len(mnist_test)}筆資料')


mnist_train = DataLoader(mnist_train, batch_size=BATCH_SIZE, shuffle=True) # 我們想要打散訓練資料的順序
mnist_test = DataLoader(mnist_test, batch_size=1)

## 來看一下我們的資料

In [ ]:
if 用離線的MNIST:
    preview_train = MNISTDataset(train=True)
else:
    preview_train = MNIST(root='mnist', download=True, transform=None, train=True)

# 實作一: 完成這個功能
indexs = [-1] * 10
# 找出第一個0-9在dataset裡的位置
for i, batch in enumerate(preview_train):
    # 把一個batch拆開, img是圖片, label是圖片的數字
    img, label = batch
    # 在這裡開始寫
    
# 輸出結果
for num, idx in enumerate(indexs):
    print(f'第一張數字 {num}的圖片出現在dataset的第 {idx}個位置')
print(indexs)

In [ ]:
for idx in indexs:
    img, label = preview_train[idx]
    plt.title(f'label: {label}')
    plt.imshow(img, cmap='gray')
    plt.show()

## 宣告損失函數&優化器

In [ ]:
# 計算輸出的損失函數(Loss function)
loss_func = CrossEntropyLoss() # 計算機率的Loss我們會用交叉熵(CrossEntropy)
# 這是優化器(Optimizer), 使得模型能更好的學習, 可以想成是學習的策略
optimizer = Adadelta(model.parameters(), lr=LR)

## 訓練

In [ ]:
# 把每個EPOCH的Loss記錄下來
losses_log = []
# 開始訓練
for epoch in trange(1, EPOCHS+1, desc='Epoch', unit='次'):
    # 忘記剛才學習的方向
    losses = 0
    optimizer.zero_grad()
    # 給模型看很多的圖
    for batch_x, batch_y in tqdm(mnist_train, desc='訓練進度', unit='batch'):
        # 把圖跟答案放到GPU
        x = batch_x.cuda()
        y = batch_y.cuda()
        # 問模型這是什麼
        predict = model(x)
        # 根據模型的回答評分
        loss = loss_func(predict, y)
        # 告訴模型哪裡錯了
        loss.backward()
        # 把所有錯的地方跟程度加起來
        losses += loss.item()
        # 模型根據上面給的方向學習
        optimizer.step()
    

    # 測試, 給模型看從沒看過的圖, 看他是真懂還是假懂
    test_losses = 0
    trues = []
    predicts = []
    with torch.no_grad():
        for batch_x, batch_y in tqdm(mnist_test, desc='測試進度', unit='張'):
            x = batch_x.cuda()
            y = batch_y.cuda()

            predict = model(x)
            test_losses += loss_func(predict, y).item()
            # 選模型認為最有可能的數字
            predict = torch.argmax(predict, dim=1, keepdim=True)
            trues += y.tolist()
            predicts += predict.tolist()
        # 記錄Loss
        losses_log.append(test_losses)
        # 印出這次Epoch的總結與統計
        print(f'EPOCH {epoch} | 訓練資料的Loss: {losses/len(mnist_train.dataset)} | 測試資料的Loss: {test_losses/len(mnist_test.dataset)}\n{classification_report(trues, predicts, labels=[l for l in range(0, 10)], digits=4)}')



## 把訓練過程的Loss 畫出來

In [ ]:
# X 軸是Epoch, Y 是對應Epoch 的Loss
plt.plot(list(range(1, EPOCHS+1)), losses_log)
plt.xticks(list(range(1, EPOCHS+1)))
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xlim(1, EPOCHS)